In [ ]:
!pip install langchain
!pip install faiss-cpu

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import tensorflow_hub as hub

from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
import pandas as pd

In [4]:
saas_data = "GPT based triaging.txt"

requests, teams = [], []

with open(saas_data, "r") as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith("Request:"):
            request = line.split("Request:", 1)[1].strip()
            request = request.strip('"') # This line removes the quotation marks
            requests.append(request)
        elif line.startswith("Team:"):
            teams.append(line.split("Team:", 1)[1].strip())

df = pd.DataFrame({
    'Request': requests,
    'Team': teams
})

df['Team'].value_counts()

Support Team                460
Finance Team                459
Product Development Team    458
Sales Team                   17
Training Team                 6
Marketing Team                5
Legal Team                    5
Security Team                 4
IT Operations Team            4
Implementation Team           2
Finance                       1
Name: Team, dtype: int64

In [5]:
df = df[(df['Team'] == 'Support Team') | (df['Team'] == 'Finance Team') | (df['Team'] == 'Product Development Team')]
df['Team'].value_counts()

Support Team                460
Finance Team                459
Product Development Team    458
Name: Team, dtype: int64

In [6]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_model = hub.load(module_url)

In [10]:
X = df['Request']
y = df['Team']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.iloc[2]

"We've identified a need for additional data visualization options within the software. Can you explore adding more chart types?"

In [ ]:
from langchain.docstore.document import Document
documents = []

for index in range(len(X_train)):
    documents.append(Document(page_content=X_train.iloc[index], metadata={'source': y_train.iloc[index]}))

documents

In [18]:
from langchain.docstore.document import Document
import faiss
import numpy as np

# Convert the content of each Document to embeddings
doc_embeddings = use_model([doc.page_content for doc in documents])

# Convert embeddings to a format FAISS can work with (float32)
doc_embeddings = doc_embeddings.numpy().astype('float32')

# Create a FAISS index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

In [23]:
def k_nearest_neighbor_search(input_sentence, k=10):
    # Convert the input sentence to embeddings
    sentence_embedding = use_model([input_sentence]).numpy().astype('float32')

    # Perform k-NN search
    distances, indices = index.search(sentence_embedding, k)

    # Retrieve and return the metadata and distances of the k-nearest documents as a list of tuples
    # nearest_documents = [(documents[i].metadata, distances[0][j]) for j, i in enumerate(indices[0])] # This returns KNN documents along with distances

    # Find the minimum distance
    min_distance = np.min(distances[0])
    # Retrieve documents with the minimum distance
    nearest_documents = [documents[i].metadata.get('source') for j, i in enumerate(indices[0]) if distances[0][j] == min_distance] # This returns a list nearest distance document.
    nearest_documents = list(set(nearest_documents))

    return nearest_documents

In [24]:
# Example usage
input_sentence = "I'm having trouble logging into my account"
nearest_documents_metadata = k_nearest_neighbor_search(input_sentence, k=10)

# Print the metadata of the nearest documents
for metadata in nearest_documents_metadata:
    print(metadata)

Support Team


In [25]:
from sklearn.metrics import classification_report

# Predict the class for each instance in X_test
y_pred = [k_nearest_neighbor_search(x) for x in X_test]

# Generate the classification report
print(classification_report(y_test, y_pred))

                          precision    recall  f1-score   support

            Finance Team       0.98      0.95      0.96        42
Product Development Team       0.84      0.88      0.86        49
            Support Team       0.83      0.81      0.82        47

                accuracy                           0.88       138
               macro avg       0.88      0.88      0.88       138
            weighted avg       0.88      0.88      0.88       138

